In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from math import radians, cos, sin, asin, sqrt, floor
from operator import truediv

# Data Imports

In [ ]:
###################################### MOON CRATER DATA ########################

# Tigeys path to moon data
craters=pd.read_csv('../input/moonproject/AstroStats_Robbins_Moon.csv', sep=',', comment='#')

###################################### GRAVITY MAP DATA ########################
titles = ['Longitude', 'Latitude', 'Height']

# Tigey's path to gravity map
grav_map = pd.read_csv('../input/moonproject/table1.xyz', header=None, names=titles, sep=" ")


#I always like to look at the headings
craters.head()
grav_map.head()

# Functions for Data Cleaning and Projection

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 1737.1
    return c * r
 
def rectangle_projection(lon, lat, z, lon1, lat1, lon2, lat2):
    """
    Turns lon/lat rectangle into standardized kilometer projection, with
    0,0 at the left middle.
 
    Args:
        lon: input the longitudinal data (pandas df)
        lat: input latitude data (pandas df)
        z: input values. either heatmap data or crater size
        lon1, lat1: lon/lat of first rectangle point
        lon2, lat2: lon/lat of second rectangle point
    """
    count = 0
    midlat = np.mean([lat1, lat2])
    centerkm = ((lon2 - lon1)/360) * lat2circ(midlat)
    lonkm = haversine(0, lat1, 0, lat2)
    d = {'X': [], 'Y': [], 'Values': []}
    df = pd.DataFrame(data=d)
    print(("-"*4 + "+"*1 + "-"*4 + "|"*1)*4 + ("-"*4 + "+"*1 + "-"*4 + "V"*1)*1 + ("-"*4 + "+"*1 + "-"*4 + "|"*1)*5) # Loading bar
    count = 0
    for i in range(0, len(lon)):
        if lon1 < lon[i] < lon2 and lat1 < lat[i] < lat2:
            x = ((lon[i] - lon1) / (lon2 - lon1)) * centerkm
            y = ((lat[i] - midlat) / 360) * 10921
            df = df.append(pd.Series([x, y, z[i]], index=df.columns),  ignore_index=True)
        print("|" * (int(((i/len(lon))) * 100) - count), end="") # Loading bar
        count = int((i/len(lon)) * 100) # Loading Bar
    print("|")
    return df, lonkm, centerkm
 
def lat2circ(lat):
  """
  Turns a given latitude into the circumference of the longitude line
  at that latitude.
  """
  return 3.14159 * 2 * 1737.1 * sin(radians(90-abs(lat)))

def dot_projection_count(xr, yr, df, g=0):
    """
    Creates a grid of points with values corresponding to how many craters
    overlap each point. Designed to be used with a heatmap to visulaize
    crater intersection.
    """
    if g > 0:
        df = df.nlargest(g, df.columns[2])
    print(("-"*4 + "+"*1 + "-"*4 + "|"*1)*4 + ("-"*4 + "+"*1 + "-"*4 + "V"*1)*1 + ("-"*4 + "+"*1 + "-"*4 + "|"*1)*5) # Loading bar
    p = []
    count = 0
    for x in range(0, xr):
        for y in range(-yr, yr):
            h=0
            for i in range(0, len(df)):
                t=sqrt((x-df.iloc[i,0])**2+(y-df.iloc[i,1])**2)
                if t<=(df.iloc[i,2])/2:
                    h=h+1
            p.append([x, y, h])
        print("|" * (int(((x/xr)) * 100) - count), end="") # Loading bar
        count = int((x/xr) * 100) # Loading Bar
    print("|")
    return pd.DataFrame(np.array(p),columns=['x','y','h'])

def dot_projection_grail(craters, grail, g=0):
    """
    Creates a grid of points with values corresponding to GRAIL values for those
    points, but only if they intersect craters. Effectively a GRAIL map of craters.
    Designed to be used with a heatmap to visualize crater GRAIL attributes.
    """
    if g > 0:
        craters = craters.nlargest(g, craters.columns[2])
    d1 = {'X': [], 'Y': [], 'Value': []}
    output = pd.DataFrame(data=d1)
    print(("-"*4 + "+"*1 + "-"*4 + "|"*1)*4 + ("-"*4 + "+"*1 + "-"*4 + "V"*1)*1 + ("-"*4 + "+"*1 + "-"*4 + "|"*1)*5) # Loading bar
    count = 0
    for index, crater in craters.iterrows():
        for index2, point in grail.iterrows():
            h = 0
            if sqrt((point['X']-crater['X'])**2+(point['Y']-crater['Y'])**2) <= crater['Values'] / 2:
                output = output.append(pd.Series([point['X'], point['Y'], point['Values']], index=output.columns),  ignore_index=True)
        print("|" * (int(((index/len(craters))) * 100) - count), end="") # Loading bar
        count = int((index/len(craters)) * 100) # Loading Bar
    print("|")
    return output

def crater_grail_average(df3, df4, b1, b2):
    """
    Finds the average GRAIL gravity change for each given crater. Designed to be
    used with a scatterplot to show how crater size affects gravity change.
    """
    d1 = {'X': [], 'Y': [], 'Diameter': [], 'Values': [], 'Baseline': []}
    output = pd.DataFrame(data=d1)
    print(("-"*4 + "+"*1 + "-"*4 + "|"*1)*4 + ("-"*4 + "+"*1 + "-"*4 + "V"*1)*1 + ("-"*4 + "+"*1 + "-"*4 + "|"*1)*5) # Loading bar
    count = 0
    for index, crater in df3.iterrows():
        c = 0 # total count of points within crater
        v = 0 # sum of GRAIL values inside crater
        cb = 0 # total count of points in baseline
        vb = 0 # sum of GRAIL values in baseline 
        for index2, point in df4.iterrows():
            dist = sqrt((point['X']-crater['X'])**2+(point['Y']-crater['Y'])**2)
            if  dist <= crater['Values'] / 2:
                v += point['Values']
                c += 1
            elif dist >= (crater['Values'] / 2) * b1 and dist <= (crater['Values'] / 2) * b2:
                vb += point['Values']
                cb += 1
        print("|" * (int(((index/len(df3))) * 100) - count), end="") # Loading bar
        count = int((index/len(df3)) * 100) # Loading Bar
        if c > 0 and cb > 0:
            output = output.append(pd.Series([crater['X'], crater['Y'], crater['Values'], v/c, vb/cb], index=output.columns),  ignore_index=True)
    print("|")
    return output

def crater_grail_average_rims(df3,df4,b1,b2):
    """
    Finds the average GRAIL gravity change for radial rings in each given crater.
    Designed to be used with a plot to show how graity changes from the center 
    to the outer parts of a crater.
    """
    d1 = {'X': [], 'Y': [], 'Diameter': [], 'Values': [], 'Baseline': []}
    output = pd.DataFrame(data=d1)
    print(("-"*4 + "+"*1 + "-"*4 + "|"*1)*4 + ("-"*4 + "+"*1 + "-"*4 + "V"*1)*1 + ("-"*4 + "+"*1 + "-"*4 + "|"*1)*5) # Loading bar
    count = 0
    for index, crater in df3.iterrows():
        c = [0]*10 # total count of points within crater
        v = [0]*10 # sum of GRAIL values inside crater
        cb = 0 # total count of points in baseline
        vb = 0 # sum of GRAIL values in baseline 
        for index2, point in df4.iterrows():
            dist = sqrt((point['X']-crater['X'])**2+(point['Y']-crater['Y'])**2) / (crater['Values']/2)
            if  dist <= 1:
                dist = floor(dist*10)
                v[dist] += point['Values']
                c[dist] += 1
            elif dist >= b1 and dist <= b2:
                vb += point['Values']
                cb += 1
        print("|" * (int(((index/len(df3))) * 100) - count), end="") # Loading bar
        count = int((index/len(df3)) * 100) # Loading Bar
        if min(c) > 0 and cb > 0:
            output = output.append(pd.Series([crater['X'], crater['Y'], crater['Values'], list(map(truediv, v, c)), vb/cb], index=output.columns),  ignore_index=True)
    print("|")
    return output

# Compile Dataframe

In [ ]:
craters_flattened, x, y = rectangle_projection(craters['LON_CIRC_IMG'], craters['LAT_CIRC_IMG'], craters['DIAM_CIRC_IMG'], 0, -10, 20, 10)
grav_flattened, x2, y2 = rectangle_projection(grav_map['Longitude'], grav_map['Latitude'], grav_map['Height'], 0, -10, 20, 10)
craters_flattened_1 = craters_flattened[craters_flattened.Values > 1]
grav_flattened.to_csv('/kaggle/working/grav_flattened.csv')
craters_flattened_1.to_csv('/kaggle/working/craters_flattened_1.csv')

# Visualize GRAIL Data for Select Craters

In [ ]:
# Find GRAIL values for the biggest 10 craters
grail_dot = dot_projection_grail(craters_flattened_1, grav_flattened, 10)
grail_dot.to_csv('/kaggle/working/grail_dot.csv')

In [ ]:
# Plot two subplots including the 10 largest craters and a respective GRAIL map
plt.figure(figsize=(10, 3.5))
plt.suptitle("Comparison of Gravity and Crater Map at 10S-10N, 160-180E")
plt.subplots_adjust(wspace=0.4, top=0.8)
plt.subplot(1, 2, 1)
plt.scatter(grail_dot['X'],grail_dot['Y'],c=grail_dot['Value'])
plt.set_cmap("gray")
plt.colorbar().set_label('mGal Gravity Change from Standard')
plt.xlabel("Distance (km)")
plt.ylabel("Distance (km)")
plt.title("Selected Crater Locations")

plt.subplot(1, 2, 2)
plt.scatter(grav_flattened['X'], grav_flattened['Y'], c=grav_flattened['Values'])
plt.set_cmap("gray")
cbar2 = plt.colorbar().set_label('mGal Gravity Change from Standard')
plt.xlabel("Distance (km)")
plt.ylabel("Distance (km)")
plt.title("GRAIL Gravity Map")

# Get Average GRAIL Values of All Craters

In [ ]:
# Find the average GRAIL values for all craters
avg_grail = crater_grail_average(craters_flattened_1, grav_flattened, 1.25, 1.5)
avg_grail.to_csv('/kaggle/working/avg_grail.csv')

In [ ]:
# Scatterplot of mean of all craters v their diameter
plt.scatter(avg_grail['Diameter'], avg_grail['Values']-avg_grail['Baseline'], s=3, c='r')
plt.xlabel('Crater Diameter (km, log scale)')
plt.ylabel('Average GRAIL Value inside Crater (mGal)')
plt.title('Crater Size v Gravity Change at 10S-10N, 160-180E')
plt.xscale('log')

# Get Radial Gravity Change within Craters

In [ ]:
# Get the radial gravity (over 10 points) inside each crater
avg_grail_radial = crater_grail_average_rims(craters_flattened_1, grav_flattened, 1.25, 1.5)
avg_grail_radial.to_csv('/kaggle/working/avg_grail_radial.csv')
avg_grail_radial.sort_values(by=['Diameter'], ascending=False)
x = np.linspace(0,10,10)*10

In [ ]:
plt.figure(figsize=(10, 3.5))
plt.suptitle("Comparison of Radial Gravity for Largest and Smallest Craters at 10S-10N, 160-180E")
plt.subplots_adjust(wspace=0.4, top=0.8)
plt.subplot(1, 2, 1)
plt.plot(x, avg_grail_radial['Values'][0])
plt.plot(x, avg_grail_radial['Values'][1])
plt.plot(x, avg_grail_radial['Values'][2])
plt.plot(x, avg_grail_radial['Values'][3])
plt.title('Radial Gravity of 4 Largest Craters')
plt.xlabel('Percent of Radius')
plt.ylabel('Gravity Deviation from Standard (mGal)')

plt.subplot(1, 2, 2)
plt.plot(x, avg_grail_radial['Values'].iloc[-1])
plt.plot(x, avg_grail_radial['Values'].iloc[-2])
plt.plot(x, avg_grail_radial['Values'].iloc[-3])
plt.plot(x, avg_grail_radial['Values'].iloc[-4])
plt.title('Radial Gravity of 4 Smallest Craters')
plt.xlabel('Percent of Radius')
plt.ylabel('Gravity Deviation from Standard (mGal)')

# Statistical Analysis

In [ ]:
c1=avg_grail[(avg_grail.Diameter >0) & (avg_grail.Diameter <10)]
c2=avg_grail[(avg_grail.Diameter >10) & (avg_grail.Diameter <45)]
c3=avg_grail[(avg_grail.Diameter >45)]

c1_m=c1['Values'].mean()
c2_m=c2['Values'].mean()
c3_m=c3['Values'].mean()

c1_std=c1['Values'].std()
c2_std=c2['Values'].std()
c3_std=c3['Values'].std()

h='.k'
plt.errorbar(5, c1_m,yerr=c1_std,fmt=h)
plt.errorbar(27.5, c2_m,yerr=c2_std,fmt=h)
plt.errorbar(60, c3_m,yerr=c3_std,fmt=h)

plt.title('Means and Std of Mean Crater Gravity v Crater Type')
plt.ylabel('Mean Gravity Deviation (mGal)')
plt.xlabel('Crater Type (Simple, Complex, Impact Basin)')